In [ ]:
import pandas as pd 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

import os
print(os.listdir("../input"))


# Atrybuty obu datasetów:

* school - szkoła ucznia ('GP' - Gabriel Pereira lub 'MS' - Mousinho da Silveira)
* sex - płeć ucznia ('F' - kobieta(female) lub 'M' - mężczyzna(male))
* age - wiek ucznia (od 15 do 22)
* address - typ adresu zamieszkania ucznia ('U' - miejski lub 'R' - wiejski)
* famsize - rozmiar rodziny ('LE3' - mniej niż 3 osoby lub 'GT3' - więcej niż 3 osoby)
* Pstatus - status rodziców ('T' - żyją wspólnie lub 'A' - oddzielnie(rozwiedzeni))
* Medu - wykształcenie matki (0 - brak, 1 - podstawowe(4th grade), 2 – from 5th to 9th grade, 3 – średnie lub 4 – wyższe)
* Fedu - wykształcenie ojca (0 - brak, 1 - podstawowe(4th grade), 2 – from 5th to 9th grade, 3 – średnie lub 4 – wyższe)
* Mjob - praca matki ('teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' lub 'other')
* Fjob - praca ojca  ('teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' lub 'other')
* reason - powód wybrania szkoły (close to 'home', school 'reputation', 'course' preference lub 'other')
* guardian - opiekun ucznia ('mother', 'father' lub 'other')
* traveltime - czas drogi do szkoły ( 1 - 10 godzin)
* studytime - tygodniowy czas nauki (1 - 10 godzin)
* failures - liczba niezaliczonych klas (n if 1<=n<3, else 4)
* schoolsup - pomoc w szkole(tak lub nie)
* famsup - pomoc dla rodziny (tak lub nie)
* paid - dodatkowe opłacane kursy (tak lub nie)
* activities - dodatkowe zajęcia (tak lub nie)
* nursery - był w żłobku (tak lub nie)
* higher - chce wyższego wykształcenia (tak lub nie)
* internet - czy ma dostęp do Internetu w domu(tak lub nie)
* romantic - czy jest w związku(tak lub nie)
* famrel - jakość relacji w rodzinie (od 1 - bardzo zła do 5 - bardzo dobra)
* freetime - czas wolny po zajęciach (od 1 -bardzo mało do 5 - bardzo dużo)
* goout - czy spędza czas z przyjaciółmi (od 1 - bardzo rzadko do 5 - bardzo często)
* Dalc - konsumpcja alkoholu w dniach roboczych (od 1 - bardzo mało(brak) do 5 - bardzo dużo)
* Walc - konsumpcja alkoholu w weekendy (od 1 - bardzo mało(brak) do 5 - bardzo dużo)
* health - obecny stan zdrowia (od 1 - bardzo słaby do 5 - wspaniały)
* absences - ilość nieobecności (od 0 do 93)

Oceny powiązane z kursami: matematyki, portugalskiego:

* G1 - ocena za pierwszy semestr (od 0 do 20)
* G2 - ocena za drugi semestr (od 0 do 20)
* G3 - ocena za cały rok (od 0 do 20, output target)

# Wczytanie danych

In [ ]:
student_mat = pd.read_csv('../input/alc-consumption-and-higher-education/student-mat.csv')
student_mat.head()

In [ ]:
student_por = pd.read_csv('../input/alc-consumption-and-higher-education/student-por.csv')
student_por.head()

Określenie do którego kursu uczeń, należy(jest to istotne po zmergowaniu obu tabel)

In [ ]:
student_mat['course']='mat'
student_por['course']='por'

Od razu też zmerguję moje dane, zanim zacznę je wizualizować.

In [ ]:
student_mat= student_mat.append([student_mat,student_por])
x = student_mat.iloc[:, [3]].values
student_mat

# Wizualizacja danych

Zacznijmy od heatmapy, pokazującej korelację zmiennych między sobą.

In [ ]:
plt.figure(figsize=(32, 30))
sns.heatmap(student_mat.corr(), annot=True)
plt.show()

Zestawienie wykresów przedstawiające ilość uczniów w zależności od kursu jaki podjęli. Wykresy przedstawiają dane wybranych szkół przez uczniów, ich płci, wieku itp.

In [ ]:
plt.figure(figsize=[15,17])
cols=['school','sex','age','Mjob','Fjob','Medu','Fedu','Dalc','Walc','paid','activities','health','absences','G1','G2','G3']
n=1
for col in cols:
    plt.subplot(4,4,n)
    sns.countplot(x=col, hue='course', edgecolor="black", data=student_mat)
    sns.despine()
    plt.title("Countplot of {} by chosen Course".format(col))
    n+=1
plt.tight_layout()
plt.show()

Ilość uczniów podzielonych ze wględu na ich oceny w zależności od wybranego kursu.

In [ ]:
plt.figure(figsize=[15,4])
sns.countplot(x='course', hue='G3',edgecolor="black", data=student_mat)
sns.despine()
plt.title("Countplot of Course by final Grade")
plt.show()

Możemy jeszcze przedstawić, ile w ogóle uczniów zapisało się na poszczególne kursy.

In [ ]:
student_mat['course'].value_counts()

In [ ]:
sns.countplot(student_mat['course'])

# Preprocessing danych.

Zobaczmy jakie typy mamy w tabeli.

In [ ]:
student_mat.dtypes

W tym miejscu konwertuję wszystkie dane tak aby były wartościami liczbowymi

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

object_type_features = student_mat.select_dtypes("object").columns

for feat_name in object_type_features: 
    student_mat[feat_name] = le.fit_transform(student_mat[feat_name])

student_mat

In [ ]:
student_mat.dtypes

Udało się, gdyż mamy same wartości całkowite.

Możemy sprawdzić, czy nie wystpują żadne nulle w naszych rzędach

In [ ]:
student_mat.isnull().sum()

Wszystkie dane są uzupełnione, więc możemy przejść dalej.

# Podział danych na dane testowe i trenujące

In [ ]:
from sklearn.model_selection import train_test_split

X = student_mat.drop('course', axis = 1)
y = student_mat['course']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Implementacja algorytmów

Algorytm 1. Random Forest Classifier

In [ ]:
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

cm = confusion_matrix(pred_rfc, y_test)
cm

In [ ]:
print("Accuracy of RandomForestClassifier : ", accuracy(cm))

Algorytm 2. MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(hidden_layer_sizes=(33,33,33), max_iter=300,random_state=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print("Accuracy of MLPClassifier : ", accuracy(cm))

Algorytm 3. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

cm =  confusion_matrix(y_test, y_pred)
cm

In [ ]:
print("Accuracy of Logistic Regression : ", accuracy(cm))

# Cross-Validation

In [ ]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=10, random_state=1, shuffle=True)

scores = cross_val_score(rfc, X, y, scoring='accuracy', cv=cv)
print('Accuracy of Random Forest Classifier after cross validation: ', np.mean(scores))

In [ ]:
scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy of MLP after cross validation: ', np.mean(scores))

In [ ]:
scores = cross_val_score(lr, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy of Logistic Regression after cross validation: ', np.mean(scores))